In [1]:
import open3d as o3d
import numpy as np
import subprocess
import os
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
input_folder="old"
output_folder="new_ply"
subprocess.run(["g++", "convert_to_pointcloud.cpp", "-o", "./a.exe"])
cpp_args = [input_folder,output_folder]
subprocess.run(["./a.exe"]+cpp_args, stdout=subprocess.PIPE, text=True)

In [6]:
def furthest_point_sampling(pcd,num_points):
    '''
    Furthest Point Sampling Algorithm
    Takes as input:
    pcd->point cloud(.ply) file
    num_points->user defined number of points for down-sampling
    Returns:new down-sampled point cloud(.ply)
    '''
    distances = pcd.compute_nearest_neighbor_distance()
    farthest_indices = np.zeros((num_points,), dtype=np.int32)
    seed = 0
    farthest_indices[0] = seed
    for i in range(1, num_points):
        seed = int(np.argmax(distances))
        farthest_indices[i] = seed
        distances[seed] = -1 
    downpcd=pcd.select_by_index(farthest_indices)
    cl, index = downpcd.remove_statistical_outlier(nb_neighbors=400,std_ratio=2)
    #outlier_cloud = downpcd.select_by_index(index, invert=True)
    return downpcd.select_by_index(index)
def outlier_remove(pcd,nb_neighbours,std_ratio):
    '''
    Removes the outlier from the point cloud
    Input:
    pcd->original point cloud
    nb_neighbours=user defined value. Should be around 10
    std_ratio=user defined value which provides the extent of outlier removal.Very high or very low values will lead to bad accuracy.Should be between:2-5
    Returns: new point cloud with outliers removed
    '''
    cl, index = pcd.remove_statistical_outlier(nb_neighbours,std_ratio)
    inlier_cloud = pcd.select_by_index(index)
    outlier_cloud = pcd.select_by_index(index, invert=True)
    return pcd.select_by_index(index)

In [10]:
'''
Sample usage without any downsampling or outlier removal
To see the original point cloud
'''
file_name="down_ply/p1.ply"
pcd=o3d.io.read_point_cloud(file_name)
o3d.visualization.draw_geometries([pcd])

In [4]:
'''
Sample usage with downsampling to num_points and outlier removal with std_ratio=5 and nb_neighbours=10
To see the original point cloud
'''
file_name="p1.ply"
num_points=20000
pcd=o3d.io.read_point_cloud(file_name)
dpcd=furthest_point_sampling(pcd,num_points)
dpcd=outlier_remove(dpcd,10,5)
print([dpcd])
o3d.visualization.draw_geometries([dpcd])

[Open3D WARNING] Read PLY failed: unable to read file: p1.ply
[PointCloud with 19426 points.]


In [3]:
def list_files(directory):
    '''
    this function reads the directory which contains the point cloud(.ply) files to be downsampled
    '''
    files = []
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    return files

In [8]:
'''
Sample usage with downsampling to num_points and outlier removal with std_ratio=5 and nb_neighbours=10
to downsample all the point cloud(.ply) files from old_folder_path to new_folder_path

old_folder_path:containing all the original point clouds that you want to down-sample
new_folder_path:the folder where you want to save all the down-sampled point clouds(.ply)
num_points=to how mnay points you want to down-sample the point cloud
'''
old_folder_path="old_ply"
new_folder_path="down_ply"
file_paths = list_files(old_folder_path)
num_points=20000
for file in file_paths:
    pcd=o3d.io.read_point_cloud(file)
    dpcd=furthest_point_sampling(pcd,num_points)
    dpcd=outlier_remove(dpcd,10,5)
    new_file_path=new_folder_path+"/"+file[len(old_folder_path)+1:]
    o3d.io.write_point_cloud(new_file_path,dpcd)
    print([dpcd])
    #o3d.visualization.draw_geometries([dpcd])


[Open3D WARNING] Read PLY failed: unable to read file: old_ply\BR236-A236_1SR_DEF_10WFP_HP-1-M50BR236-A236_1SR_DEF_10WFP_HP-1-M50_08081010.ply
[PointCloud with 19340 points.]
[Open3D WARNING] Read PLY failed: unable to read file: old_ply\p1.ply
[PointCloud with 19426 points.]
